In [1]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext, SQLContext, Row, SparkConf

# Start Spark Context and sqlContext

In [8]:
conf = SparkConf().setAppName("nasa_data_analysis").setMaster("local")
sc = SparkContext(conf=conf)

In [9]:
sqlContext = SQLContext(sc)

# Load Data 

In [10]:
rdd_data = sc.textFile("../data/access_log_Aug95")

# Take first element to see the structure of line 

In [12]:
sample = rdd_data.take(1)[0]
sample

'in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839'

# Stop Context 

In [6]:
sc.stop()